In [ ]:
import sys,os,time

import tables as tb
import yaml
import numpy as np
from matplotlib import colors
norm = colors.LogNorm()
import matplotlib.pyplot as plt

import tjmonopix.tjmonopix
import tjmonopix.scans.source_scan as source_scan
import tjmonopix.scans.th_scan as th_scan

DUtoe=20

# Electrical tests

In [ ]:
chip=tjmonopix.tjmonopix.TJMonoPix(
    conf="/home/user/workspace/tjmonopix/tjmonopix_30/tjmonopix/tjmonopix_mio3.yaml",
    no_power_reset=False)
chip.init()

In [ ]:
# reset configuration
chip.default_conf()
chip.reset_ibias()

In [ ]:
####### CONFIGURE THE FRONT END ######
# INPUT baseline
chip.set_vreset_dacunits(35,1) #1V
# OUTPUT BASELINE
chip.set_icasn_dacunits(0,1) #4.375nA # approx 1.084V at -3V backbias, 600mV at 0V backbias
# INOUT RESET rate
chip.set_ireset_dacunits(2,1,1) #270pA, HIGH LEAKAGE MODE, NORMAL SCALING, 0 = LOW LEAKAGE MODE, SCALING*0.01
# OUTPUT RESET rate
chip.set_ithr_dacunits(5,1) #680pA
# BIASING OF THE DISCRIMINATOR (AND THE THRESHOLD)
chip.set_idb_dacunits(20,1) #500nA
# DC current of Amp, OUTPUT RISE TIME
chip.set_ibias_dacunits(20,1) #500nA
chip.write_conf()

In [ ]:
#### (optional) check analog output
chip.enable_analog(col="all")
chip.set_inj_all(inj_n=0)  ## run this cell with inj_n=1 to stop injection
chip.inject()

In [ ]:
#### enable pixels and mask noisy pixels
chip.set_monoread()
noisy=chip.auto_mask()

mask=chip.get_pixel_status(mask=None,mode="all")
plt.imshow(chip.get_pixel_status(mask=mask,mode="preamp")[chip.fl_n,:,:],
           interpolation="none",origin="bottomleft")
plt.title("Enabled pixel = yellow");

In [ ]:
### check power status (VDDD < several mA, VDDP roughly 80mA)
chip.get_power_status()

In [ ]:
### select one pixel
col=50
row=50
chip.enable_injection(chip.fl_n, col, row)
#chip.enable_pixel(chip.fl_n,col,row) 
#chip.enable_hitor(chip.fl_n,col,row)
mask=chip.get_pixel_status(mask=None,mode="all")
print "preamp of [%d,%d]:"%(col,row),chip.get_pixel_status(mask=mask,mode="preamp")[chip.fl_n, col, row]
print "inj of [%d,%d]:"%(col,row),chip.get_pixel_status(mask=mask,mode="inj")[chip.fl_n, col, row]
print "hitor of [%d,%d]:"%(col,row),chip.get_pixel_status(mask=mask,mode="mon")[chip.fl_n, col, row]

In [ ]:
### (optional) inject 1 pixel. it is good idea to run this once/twice.
chip.set_inj_all(inj_n=1000,inj_phase=0)
#chip.reset_ibias()
chip.set_monoread()
chip.inject()
time.sleep(1)
dat=chip.interpret_data(chip['fifo'].get_data())
print "n of data", len(dat)
if len(dat)!=0:
    uni,cnt=np.unique(dat[["col",'row']], return_counts=True)
    arg=np.argsort(cnt)
    for a in arg[::-1]:
        print uni[a],cnt[a]

In [ ]:
### S-curve
chip.set_inj_all(inj_n=1000,inj_phase=0)
reload(th_scan)
scan=th_scan.ThScan(chip)
scan.start(rowlist=np.arange(row,row+1,1), ## list of rows
           collist=np.arange(col,col+1,1), ## list of columns
           n_mask_col=1,                   ## maximum number of columns which are injected at once
           injlist=np.arange(0,35)         ## scan values of test pulse amplitude in DU
           )                               ## do not change
scan.analyze(scan.output_filename)
scan.stop()
fraw=scan.output_filename+'.h5'
fev=fraw[:-7]+'ev.h5'

with tb.open_file(fev) as f:
    cnts=f.root.Cnts[:]
    inj=f.root.ScurveFit.attrs.injlist
    fit=f.root.ScurveFit[:]
fig,ax=plt.subplots(ncols=1,nrows=1,sharey=True)
ax=[ax]
cnts=cnts[np.bitwise_and(cnts['col']==col,cnts['row']==row)]
fit=fit[np.bitwise_and(fit['col']==col,fit['row']==row)][0]
cnt=np.zeros(len(inj))
for c in cnts:
    cnt[np.argwhere(inj==c["inj"])[0][0]]=c['cnt']
x,y=tjmonopix.analysis.utils.scurve_from_fit(th=inj,n=500,
        reverse=False,A_fit=fit["A"],mu_fit=fit["mu"],sigma_fit=fit['sigma'])
ax[0].plot(inj,cnt,"o")
ax[0].plot(x,y,"-")
ax2=ax[0].twiny()
ax[0].set_xbound(np.min(inj),np.max(inj))
ax2.set_xbound(np.min(inj)*DUtoe,np.max(inj)*DUtoe)
ax[0].set_xlabel("Injection [DU]")
ax2.set_xlabel("Signal [e]")
print "fit mu=%.3e(%.4fDU)"%(fit["mu"]*DUtoe,fit["mu"]),
print "sigma=%.3e(%.4fDU)"%(fit["sigma"]*DUtoe,fit["sigma"])

In [ ]:
## Threshold scan 1col=~1.5min
chip.set_inj_all(inj_n=100,inj_phase=0)
reload(th_scan)
scan=th_scan.ThScan(chip)
scan.start(rowlist=np.arange(0,224,1), ## list of rows
           collist=np.arange(0,112,10), ## list of columns change here (0,112,1) for full scan
           n_mask_col=1,                   ## max number of columns to be injected
           injlist=np.arange(0,35),        ## scan values of test pulse amplitude in DU
           debug=4)                        
scan.analyze(scan.output_filename)
scan.stop()
fraw=scan.output_filename+'.h5'
fev=fraw[:-7]+'ev.h5'

In [ ]:
# superinposed s-curves

with tb.open_file(fev) as f:
    cnts=f.root.Cnts[:]
    inj=f.root.ScurveFit.attrs.injlist

fig,ax=plt.subplots(ncols=1,nrows=1,sharey=True)
hist=ax.hist2d(cnts['inj']*DUtoe,cnts['cnt'], bins=[inj*DUtoe,np.arange(100)],norm=norm)
cb=fig.colorbar(hist[3], ax=ax)
cb.set_label("# of pixels")
ax.set_ylabel("Count")
ax.set_xlabel("Signal [e]");
print "%d pixels sends data"%len(np.unique(cnts[["col","row"]]))

In [ ]:
# threshold, noise distribution

with tb.open_file(fev) as f:
    fit=f.root.ScurveFit[:]
fig,ax=plt.subplots(ncols=2,nrows=1,sharey=True)
ax[0].hist(fit['mu']*DUtoe,bins=np.arange(0,30,0.5)*DUtoe,histtype="step")
ax[1].hist(fit['sigma']*DUtoe,bins=np.arange(0,3,0.05)*DUtoe,histtype="step")
ax[0].set_ylabel("# of pix")
ax[0].set_xlabel("Threshold [e]")
ax[1].set_xlabel("ENC [e]");

In [ ]:
## noise occupancy
reload(source_scan)
scan=source_scan.SourceScan(dut=chip)
scan.start(scan_time=100,                                ## scan time in seconds
           with_mon=False,with_rx1=False,with_tlu=False) ## do not change
fhit=scan.analyze(scan.output_filename,event_build="none",clusterize=False)
scan.stop()
with tb.open_file(fhit) as f:
    hits=f.root.Hits[:]
fig,ax=plt.subplots(ncols=2,nrows=1)
hist=ax[0].hist2d(hits['col'],hits['row'],bins=[np.arange(0,112),np.arange(0,224)]);
fig.colorbar(hist[3],ax=ax[0])
ax[1].hist(np.reshape(hist[0],[-1]),histtype="step",bins=20)
ax[1].set_yscale("log")
ax[0].set_title("Hit map");
ax[1].set_ylabel("# of pixels");
fig.tight_layout()

# Source scan

In [ ]:
### (optional) load setting from confiuration file
with open("Mod1W7R19.yaml") as f:
    conf=yaml.safe_load(f)
#chip=tjmonopix.tjmonopix.TJMonoPix(
#    conf=conf['dut']['file'],
#    no_power_reset=True)
#chip.init()    
    
chip.default_conf()
chip.switch_flavor(conf["dut"]['flavor'])
chip.enable_pixel(chip.fl_n,col=-1,row=-1,mask=conf['mask'])
chip.reset_ibias()

chip.set_vreset_dacunits(conf['dut']['vreset_dacunits'],1) #
chip.set_icasn_dacunits(conf['dut']['icasn_dacunits'],1) 
chip.set_ireset_dacunits(conf['dut']['ireset_dacunits']['value'],conf['dut']['ireset_dacunits']['mode'],1)
chip.set_ithr_dacunits(conf['dut']['ithr_dacunits'],1)
chip.set_idb_dacunits(conf['dut']['idb_dacunits'],1)
chip.set_ibias_dacunits(conf['dut']['ibias_dacunits'],1)
chip.reset_ibias()

In [ ]:
## Fe
## online monitor
## % start_online_monitor ../tjmonopix/online_monitor.yaml

reload(source_scan)
scan=source_scan.SourceScan(dut=chip)
scan.start(scan_time=1000,                                ## scan time in seconds
           with_mon=False,with_rx1=False,with_tlu=False) ## do not change
fcl=scan.analyze(scan.output_filename,event_build="token",clusterize=True)
scan.stop()

In [ ]:
with tb.open_file(fcl) as f:
    dat=f.root.Clusters[:]
tmp=dat[np.bitwise_and(dat['seed_column']==col,dat['seed_row']==row)]
bins=np.arange(0,0x41)
plt.hist(tmp['charge'],bins=bins);
plt.hist(tmp['charge'][tmp['n_hits']==1],bins=bins);

In [ ]:
## Sr with TLU
## tlu
## % pytlu -ie CH0 -io CH0 -th 30 -t 0 --timeout 0 -b 15 
reload(source_scan)
scan=source_scan.SourceScan(dut=chip)
scan.start(scan_time=100,
    with_mon=False,with_rx1=False,with_tlu=True) ## do not change
fcl=scan.analyze(scan.output_filename,event_build="token",clusterize=True)
scan.stop()
fhit=scan.output_filename[:-4]+'hit.h5'